# Lorenz (1963) system

<img src="figButterflyEffect.jpg" width=350 height=300 />

Code to simulate the 3-equation system of Lorenz (1963). <br>
Vincent Verjans, vverjans3@gatech.edu

System of equations:
$$
\begin{align}
\begin{cases}
\frac{\partial X}{\partial t} = -\sigma X + \sigma Y\\
\frac{\partial Y}{\partial t} = (r-Z) X - Y \\
\frac{\partial Z}{\partial t} = X Y - b Z\\
\end{cases}
\end{align}
$$
$X \sim$ intensity of the convective motion <br>
$Y \sim$ temperature difference between ascendant and descending currents <br>
$Z \sim$ distortion of the vertical temperature profile from linearity (>0 implies strongest gradients near the boundaries) <br>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt

First: define the time stepping scheme

In [ ]:
### Time stepping ###
deltat = 0.01
ntot   = 6113
timing  = np.arange(0,ntot*deltat,deltat)

Choose the parameter values. Lorenz (1963) used the combination $\sigma=10$, $b=8/3$, $r=28$. For these values of $\sigma$ and $b$, the critical Rayleigh number occurs when $r \leq 24.74$.

In [ ]:
### Parameters ###
sigma  = 10.0
smallr = 28.0
smallb = 8/3

Next, we choose our initial values for $X, Y, Z$, and we initialze the arrays that will hold the simulated values.

In [ ]:
### Initial conditions ###
initx      = 0.0
inity      = 0.1
initz      = 0.0
xvalues    = np.zeros(ntot)
yvalues    = np.zeros(ntot)
zvalues    = np.zeros(ntot)
xvalues[0] = initx
yvalues[0] = inity
zvalues[0] = initz

Now, we define the system of Equations of Lorenz (1963). Notice that this code snippet is simply the translation of the Equations shown above into code.

In [ ]:
def systemLorenz1963(xx,yy,zz,params):
    '''
    Eqs. (25,26,27) of Lorenz (1963)
    '''
    sigmavalue = params[0]
    rvalue     = params[1]
    bvalue     = params[2]
    dxdt = -1*sigmavalue*xx+sigmavalue*yy
    dydt = (rvalue-zz)*xx-yy
    dzdt = xx*yy-bvalue*zz
    return(dxdt,dydt,dzdt)

Below, we define a function that solves a system of 3 PDEs with the Runge Kutta 4 method. This function can be used to solve our non-linear system of equations at each time step.

In [ ]:
def rk4_xyz(xx,yy,zz,step,func,params):
    '''
    Runge-Kutta 4 method for function of x,y,z and independent of time
    '''
    k1x,k1y,k1z = func(xx,yy,zz,params)
    k2x,k2y,k2z = func(xx+step*k1x/2,yy+step*k1y/2,zz+step*k1z/2,params)
    k3x,k3y,k3z = func(xx+step*k2x/2,yy+step*k2y/2,zz+step*k2z/2,params)
    k4x,k4y,k4z = func(xx+step*k3x,yy+step*k3y,zz+step*k3z,params)
    
    stepx = 1/6*(k1x+2*k2x+2*k3x+k4x)*step
    stepy = 1/6*(k1y+2*k2y+2*k3y+k4y)*step
    stepz = 1/6*(k1z+2*k2z+2*k3z+k4z)*step
    return(stepx,stepy,stepz)

All the ingredients are there to start iteratively looping through time!

In [ ]:
### Loop ###
for ii in range(ntot-1):
    
    ### Solve steps in x,y,z ###
    stepx,stepy,stepz = rk4_xyz(xvalues[ii],yvalues[ii],zvalues[ii],
                                step=deltat,func=systemLorenz1963,
                                params=[sigma,smallr,smallb])
    
    ### Update x,y,z ###
    xvalues[ii+1]=xvalues[ii]+stepx
    yvalues[ii+1]=yvalues[ii]+stepy
    zvalues[ii+1]=zvalues[ii]+stepz

We extract from our results the list of relative maxima in $Z$ (the variable $M$ in the Lorenz (1963) paper).

In [ ]:
### Get relative maxima in Z ###
relMaxZ = [zvalues[ii] for ii in range(1,ntot-1) if(zvalues[ii]>zvalues[ii-1] and zvalues[ii]>zvalues[ii+1])]

And finally, we plot our results to reproduce the figures of Lorenz (1963).

In [ ]:
### Plotting ###
fig = plt.figure(figsize=[12,10])
ax  = plt.subplot(2,2,1)
ax.scatter(yvalues,zvalues,marker='o',c='b',s=1)
ax.tick_params(axis='both',which='major',labelsize=13)
ax.set_xlabel('Y',fontsize=14),ax.set_ylabel('Z',fontsize=14)
ax  = plt.subplot(2,2,2)
ax.scatter(yvalues,xvalues,marker='o',c='b',s=1)
ax.tick_params(axis='both',which='major',labelsize=13)
ax.set_xlabel('Y',fontsize=14),ax.set_ylabel('X',fontsize=14)
ax  = plt.subplot(2,2,3)
ax.scatter(relMaxZ[0:-1],relMaxZ[1:],marker='o',c='k',s=30)
ax.tick_params(axis='both',which='major',labelsize=13)
ax.set_xlabel(r'$M_{n}$',fontsize=14),ax.set_ylabel(r'$M_{n+1}$',fontsize=14)
fig.tight_layout()